# Experimentation

In [1]:
from rstt import Ranking
from rstt.stypes import SPlayer
import pandas as pd
from scipy import stats


def evaluate(data: pd.DataFrame, reference: Ranking, trained: Ranking, population: list[SPlayer], percentiles: list[int]=[5, 10, 50]):
    row = {}
    
    # rank correlation
    row['KendallTau'] = stats.kendalltau(reference[population], trained[population]).statistic

    # rank difference
    rank_diffs = [reference[p]- trained[p] for p in population]
    row['Overrate'] = abs(min(rank_diffs))
    row['Underrate'] = max(rank_diffs)

    # accuracy
    for percentile in percentiles:
        threshold = max(int(len(population) // (100/percentile)), 1)
        row[f'{percentile}pr accuracy'] = len(set(reference[:threshold]).intersection(set(trained[:threshold]))) / threshold

    data = pd.concat([data, pd.DataFrame([row])], axis=0, ignore_index=True)
    return data

In [2]:
from rstt import BasicPlayer, BasicElo, LogSolver, BTRanking, RoundRobin

from simulation import settings, experiments, metrics

# parameters
samples = 100
NBPlayers = 60

# data 
COLUMNS = ['KendallTau', 'Underrate', 'Overrate', '5pr accuracy', '10pr accuracy', '50pr accuracy']
RESULTS: dict[str, pd.DataFrame] = {}

exp_name = 'FullRoundRobin'
RESULTS[exp_name] = pd.DataFrame(columns=COLUMNS, dtype='float')
for i in range(samples):
    # test ranking
    elo = BasicElo(f'Elo-{exp_name}')
    
    # simulation model
    model = settings.baseModel(region=1, nb=NBPlayers)
    
    # data set production
    data = experiments.regional_round_robins(f'{exp_name}-Round-Robin',
                                             seeding=model.groundtruth,
                                             population=[model.ecosystem.teams()],
                                             solver=model.solver)
    # elo update
    elo.update(**data)

    # analysis
    RESULTS[exp_name] = evaluate(RESULTS[exp_name], model.groundtruth, elo, model.groundtruth.players())

# show summary
RESULTS[exp_name].describe()

,KendallTau,Underrate,Overrate,5pr accuracy,10pr accuracy,50pr accuracy
count,100.000000,100.0000,100.000000,100.000000,100.000000,100.000000
mean,0.901887,8.3000,8.500000,0.803333,0.860000,0.949667
std,0.016107,1.9566,2.185813,0.195889,0.110402,0.027826
min,0.851977,5.0000,4.000000,0.333333,0.500000,0.866667
25%,0.891525,7.0000,7.000000,0.666667,0.833333,0.933333
50%,0.900565,8.0000,8.000000,0.666667,0.833333,0.966667
75%,0.913277,10.0000,10.000000,1.000000,1.000000,0.966667
max,0.936723,16.0000,13.000000,1.000000,1.000000,1.000000


In [3]:
from project import scene

exp_name = 'Regions'

# store metrics
for region in scene.Region:
    RESULTS[f'{exp_name}-{region}'] = pd.DataFrame(columns=COLUMNS, dtype='float')
RESULTS[exp_name]  = pd.DataFrame(columns=COLUMNS, dtype='float')

for i in range(samples):
    elo2 = BasicElo(f'Elo-{exp_name}')
    
    model = settings.baseModel(len(scene.Region), 10)
    data = experiments.regional_round_robins(f'{exp_name}-Round-Robin',
                                             seeding=model.groundtruth,
                                             population=[model.ecosystem.teams(region) for region in model.ecosystem.regions()],
                                             solver=model.solver)
    elo2.update(**data)
    
    # analysis
    for region in model.ecosystem.regions():
        RESULTS[f'{exp_name}-{region}'] = evaluate(RESULTS[f'{exp_name}-{region}'], model.groundtruth, elo2, model.ecosystem.teams(region))
    RESULTS[exp_name]  = evaluate(RESULTS[exp_name], model.groundtruth, elo2, model.ecosystem.teams())

# show summary
RESULTS[exp_name].describe()

,KendallTau,Underrate,Overrate,5pr accuracy,10pr accuracy,50pr accuracy
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.713345,20.020000,19.800000,0.473333,0.613333,0.865000
std,0.045002,4.087379,4.084956,0.260277,0.173884,0.046511
min,0.576271,12.000000,11.000000,0.000000,0.166667,0.733333
25%,0.686723,17.000000,17.000000,0.333333,0.500000,0.833333
50%,0.713559,19.000000,20.000000,0.333333,0.666667,0.866667
75%,0.749435,22.000000,23.000000,0.666667,0.666667,0.900000
max,0.816949,37.000000,36.000000,1.000000,1.000000,0.966667


In [6]:
exp_name = 'SkilledRegions'

for region in scene.Region:
    RESULTS[f'{exp_name}-{region}'] = pd.DataFrame(columns=COLUMNS, dtype='float')
RESULTS[exp_name]  = pd.DataFrame(columns=COLUMNS, dtype='float')

for i in range(samples):
    # test ranking
    elo3 = BasicElo(f'Elo-{exp_name}')
    
    # simulation model
    model = settings.regionalSkills(region=len(scene.Region),
                                    nb=10,
                                    mus=[val for val in range(750, 2500, 250)],
                                    sigmas=[300  for _ in scene.Region])
    
    # data set production
    data = experiments.regional_round_robins(f'{exp_name}-Round-Robin',
                                             seeding=model.groundtruth,
                                             population=[model.ecosystem.teams(region) for region in model.ecosystem.regions()],
                                             solver=model.solver)
    # elo update
    elo3.update(**data)

    # analysis
    for region in scene.Region:
        RESULTS[f'{exp_name}-{region}'] = evaluate(RESULTS[f'{exp_name}-{region}'], model.groundtruth, elo3, model.ecosystem.teams(region))
    RESULTS[exp_name]  = evaluate(RESULTS[exp_name], model.groundtruth, elo3, model.groundtruth.players())

# show summary
RESULTS[exp_name].describe()

,KendallTau,Underrate,Overrate,5pr accuracy,10pr accuracy,50pr accuracy
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.316791,38.960000,38.320000,0.220000,0.310000,0.641667
std,0.037314,3.776375,4.079711,0.207708,0.120837,0.047230
min,0.236158,29.000000,30.000000,0.000000,0.000000,0.533333
25%,0.285876,36.750000,35.750000,0.000000,0.166667,0.600000
50%,0.318644,39.000000,38.000000,0.333333,0.333333,0.633333
75%,0.341525,41.250000,41.000000,0.333333,0.333333,0.666667
max,0.397740,48.000000,49.000000,0.666667,0.666667,0.766667


In [17]:
RESULTS[f'{exp_name}-{scene.Region.L}'].describe()

,KendallTau,Underrate,Overrate,5pr accuracy,10pr accuracy,50pr accuracy
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000
mean,0.733333,-2.090000,38.050000,0.110000,0.110000,0.28000
std,0.114755,2.174671,4.283808,0.314466,0.314466,0.15042
min,0.377778,-7.000000,29.000000,0.000000,0.000000,0.00000
25%,0.677778,-4.000000,35.750000,0.000000,0.000000,0.20000
50%,0.733333,-2.000000,38.000000,0.000000,0.000000,0.20000
75%,0.822222,-0.750000,41.000000,0.000000,0.000000,0.40000
max,1.000000,3.000000,49.000000,1.000000,1.000000,0.60000


In [8]:
lcp = elo3.fit(model.ecosystem.teams(scene.Region.LCP))

In [9]:
lcp.plot()

----------- Standing -----------
   0.       William Bowens       1580
   1.    Heather Hickerson       1548
   2. Christopher Mcdaries       1544
   3.     Dawn Reinsfelder       1526
   4.        Tommy Pacheco       1490
   5.          Keith Ellis       1474
   6.            Frank Joy       1472
   7.         Greg Crazier       1472
   8.        Charlene Hart       1454
   9.            Ivan Byrd       1435


In [11]:
glcp = model.groundtruth.fit(lcp.keys())

In [12]:
glcp.plot()

----------- Standing -----------
   0.    Heather Hickerson       2173
   1. Christopher Mcdaries       2149
   2.       William Bowens       2052
   3.            Frank Joy       2020
   4.     Dawn Reinsfelder       2004
   5.        Tommy Pacheco       1906
   6.        Charlene Hart       1892
   7.          Keith Ellis       1649
   8.         Greg Crazier       1576
   9.            Ivan Byrd       1526
